## Notebook: General_Database_Check

#### Abstract
<font color=blue> The purpose of this notebook is to run a generic database health check. This does not look for specific archetecture pre-requisites per se. The notebook does try to gather data for general healh, best practices, and key performance indicators. Various SQL was taken from other Jupyter Notebooks written by Ember Crooks or other performance and tuning articles, blogs, and resources. Various links to these methods are at the end of the notebook in the "Credit and References" section. </font>

#### To Use This Notebook
<font color=blue> 
    
- If the libraries have not been installed, you will need to manually excute the *Install Notebook Prerequisites* section and restart the kernel.
    
- At a minimum you will need to manually *Import the Notebook Prerequisites* to load the libraries needed (this needs to be done only once after the notebook is opened that specific time).
    
- A variable file with connectivity information is expected to be held in the parent directory.
    
- Manually run the *Establish Database Connection*, type the password, and hit enter. 
    
- Once done you can select the next section and choose "Cells" from the menu bar and then "Run all Below".
    
- Note: The SQL/Code cells are hidden to make it easier to read the generated output. You can turn that off and on in the next section. I reccomend exposing the SQL untill the whole notebook runs and then I would toggle off the SQL.
</font>

#### Printing/Saving a Report
<font color=blue> 
The native Jupyter Notebook PDF generator seems to not respect hidden code cells and some data formatting is pretty ... awful. It may be more beneficial to hide the code cells and use the print function of your browser. You should be able to adjust to lanscape mode and save as a PDF.
</font>



---

## Code Toggle

<font color=blue> 
The code cells containing Python/SQL for this Jupyter notebook is hidden by default to make the notebook easier to read. To see view the queries used in this notebook, you will need to toggle on/off the raw code. <br>
    
<i>It is suggested that you expand all cells on your first run, then scroll back up to this section and hide the code blocks. This will make the report easier to read.</i>
</font>

In [ ]:
## Package prerequisites so we can have code toggle so early in the notebook.
from IPython.display import display, HTML, Markdown

## Hide code cells
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<i><b> Toggle on/off the raw code by clicking <a href="javascript:code_toggle()">here</a>.</b></i>''')

---

## Install and Import Notebook Package Prerequisites

<font color=blue> 
If you have not installed the following libraries in Python already (command line or via a notebook),
you should run the following code block. These packages are required so the notebook can work.
</font>

In [ ]:
# If this notebook is running on a Windows based machine with a Db2 client, you will need to run
# the following to allow you to run Db2 commands (i.e. db2exfmt) not basic SQL

import sys,os,os.path
os.environ['IBM_DB_HOME']='/Applications/dsdriver'

In [ ]:
# Load importlib to give yourself the ability to import details from Python to see if certian libraries exist.
# !pip3 install importlib     #Import library
import importlib

# The following will check for ibm_db_sa as a litmus test. If it exists, we assume the other libraries are installed.
# If it does not exist, install the libraries.

spec = importlib.util.find_spec("ibm_db_sa")
if spec is None:
    print("Installing prerequisites.")
    # Dependancy: ipython-sql -> sqlalchemy -> ibm_db_sa -> ibm_db
    !pip3 install ipython-sql #SQL Magic (%sql) using SQLAlchemy connect strings
    !pip3 install ibm_db      #Db2 API for Python
    !pip3 install ibm_db_sa   #Db2 adaptor for SQLAlchemy
    !pip3 install matplotlib  #Plotting library for visualizations
    !pip3 install pandas      #Data analysis and manipulation
    
    # Unnofficial community extentions, creates a new tab in Jupyter Notebook
    # Uncheck "disable configuration for nbextensions without explicit compatibility" within new tab
    !pip3 install https://github.com/ipython-contrib/jupyter_contrib_nbextensions/tarball/master
    !jupyter contrib nbextension install --user
    print()
else:
    print("Prerequisites already installed, moving to import them.")



In [ ]:
# Import necessary modules and load SQL Magic
import ibm_db
import ibm_db_sa
import sqlalchemy
%load_ext sql
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib.dates as mdates
from datetime import datetime
import pandas as pd
from IPython.display import display, HTML, Markdown
#import nbextensions
%matplotlib inline
import getpass

# Configure SQL Magic in a few nice ways
%config SqlMagic.style = 'MSWORD_FRIENDLY'
pd.set_option('display.max_rows', 4096)
pd.set_option('display.max_columns', 4096)

In [ ]:
%%html
<!-- Formatting for markup tables within the notebook -->
<style>
table {float:left}
</style>

In [ ]:
%%HTML
<!-- Formatting for data tables within the notebook -->
<style type="text/css">
table.dataframe td, table.dataframe th {
    text-align:left !important;
    border: 1px  black solid !important;
  color: black !important;
}
</style>

---

## Establish Target Database Connection

<font color=blue> 
Much of this notebook is dependant on connectivity information stored in a <i>ember_variables.py</i> file. This file can be found in 1Password. The file should be placed in the root directory of this notebook (i.e. one level up).
    
To connect to a target database, you will need to expand the code block and provide connections details including a schema name that will be used by SQL later in the notebook.
</font>

In [ ]:
# Define filename for passwords
filename = 'connection_variables.py'
# source the file
%run ./$filename

In [ ]:
# Connect to target Db2 database

# Connection Details
inst=project_inst['PROD']
user=project_user['PROD']
host=project_host['PROD']
schema='SCHEMANAME'  # Not necessary for connection, but used later to focus target of SQL
db=project_db['PROD']
port=project_port['PROD']

# Prompt for Password
password = getpass.getpass('Enter password for '+user)
import urllib
password = urllib.parse.quote(password) #Use to handle special characters

# Connection String
%sql db2+ibm_db://$user:$password@$host:$port/$db


---

## Server Level Details

<font color=blue>
From this perspective we are trying to get an idea of the size and shape of the database server. We are answering questions like "is this a Linux box", "how much processing power do we have", etc.
</font>

In [ ]:
%%sql db_server << 
SELECT
    HOST_NAME,
    OS_NAME AS OS,
    OS_FULL_VERSION AS OS_VERSION,
    OS_ARCH_TYPE,
    CPU_ONLINE AS CPU,
    CPU_CORES_PER_SOCKET AS CORES_PER_SOCKET,
    MEMORY_TOTAL / 1024 AS MEMORY_GB,
    MEMORY_SWAP_TOTAL / 1024 AS SWAP_GB,
    MEMORY_SWAP_FREE / 1024 AS SWAP_FREE_GB
FROM TABLE(SYSPROC.ENV_GET_SYSTEM_RESOURCES()) AS T
WITH UR

In [ ]:
display(Markdown("#### Server Details"))
display(Markdown("Tip: Is swap space at least the same size of configured memory? (This does not apply to a containerized database). Rule of thumb is swap is at least 1:1 match to RAM and can be 1.5-2x on highly active systems. If your swap space is consistantly highly utilized (i.e. SWAP_FREE_GB) is less than 10%, you may need to increase swap space."))
db_server_df=db_server.DataFrame()
db_server_df.columns=db_server_df.columns.str.upper()
db_server_df

---

## Db2 Version and Licensing

<font color=blue> 
In this section we are trying to deduce what level of Db2 we are running, if we are licensed properly, and if we are properly licensed for features we may be using.
</font>

In [ ]:
%%sql db_ver << 
SELECT 
    INST_NAME, 
    SERVICE_LEVEL, 
    BLD_LEVEL, 
    FIXPACK_NUM 
FROM SYSIBMADM.ENV_INST_INFO 
WITH UR

In [ ]:
display(Markdown("#### Db2 Database Version"))
display(Markdown("Tip: If you see the word SPECIAL in the build level (BLD_LEVEL), this is a special build or fixpack IBM provided to address a HIPER APAR or some specific bug you discovered that required an edit in Db2 code. If you plan on an upgrade, you may want to check with IBM to confirm you can jump from your special build to a normal fixpack version."))
db_ver_df=db_ver.DataFrame()
db_ver_df.columns=db_ver_df.columns.str.upper()
db_ver_df

In [ ]:
%%sql db_license << 
SELECT
	INSTALLED_PROD AS PRODUCT,
	INSTALLED_PROD_FULLNAME AS PROD_FULLNAME,
	PROD_RELEASE AS VERSION,
	LICENSE_INSTALLED,
	CASE
    WHEN LICENSE_TYPE IS NULL THEN 'NO LICENSE INSTALLED'
    ELSE LICENSE_TYPE
  END AS LICENSE_TYPE,
	CASE
    WHEN LICENSE_INSTALLED='Y' THEN '<-- ACTIVE LICENSE'
	  ELSE NULL
	END	AS STATUS
FROM SYSIBMADM.ENV_PROD_INFO
WITH UR

In [ ]:
display(Markdown("#### Db2 Licensing"))
db_license_df=db_license.DataFrame()
db_license_df.columns=db_license_df.columns.str.upper()
db_license_df

In [ ]:
%%sql db_feature <<
SELECT
	FEATURE_NAME AS DB2_FEATURE,
	FEATURE_FULLNAME AS FEATURE_FULLNAME,
	LICENSE_INSTALLED,
	PRODUCT_NAME AS PRODUCT,
	FEATURE_USE_STATUS
FROM SYSIBMADM.ENV_FEATURE_INFO
WITH UR

In [ ]:
display(Markdown("#### Db2 Feature and License Check"))
display(Markdown("If there are no licensable features in use, this will return and empty table."))
db_feature_df=db_feature.DataFrame()
db_feature_df.columns=db_feature_df.columns.str.upper()
db_feature_df

---

## Instance Configuration

<font color=blue>
In this section we are looking at how the overall database instance is configured. Whatever settings are in place here apply to all databases within the instance.<br>
    
We are looking for details like how long Db2 has been up and running and what best practices may or may not be in place.
</font>

In [ ]:
%%sql instance_uptime <<
SELECT 
    VARCHAR_FORMAT(DB2START_TIME, 'YYYY-MM-DD HH24:MM:SS') AS INSTANCE_ACTIVATION
    FROM TABLE(MON_GET_INSTANCE(-2))
WITH UR

In [ ]:
display(Markdown("#### Db2 Instance Start time"))
instance_uptime_df=instance_uptime.DataFrame()
instance_uptime_df.columns=instance_uptime_df.columns.str.upper()
instance_uptime_df

In [ ]:
%%sql db_env_settings <<
SELECT 
  REG_VAR_NAME AS DB2SET_PARM, 
  REG_VAR_VALUE AS DB2SET_VALUE
FROM TABLE(ENV_GET_REG_VARIABLES(-1, 1)) 
WHERE REG_VAR_NAME IN ('DB2COMM','DB2AUTH','DB2_USE_ALTERNATE_PAGE_CLEANING','DB2_PARALLEL_IO','DB2_RESTORE_GRANT_ADMIN_AUTHORITIES','AUTOSTART')
WITH UR

In [ ]:
display(Markdown("#### Common Db2 Environment Settings To Review For Best Practice"))
db_env_settings_df=db_env_settings.DataFrame()
db_env_settings_df.columns=db_env_settings_df.columns.str.upper()
db_env_settings_df

| Parameter   | Note |
| :----------- | :----------- |
| DB2_PARALLEL_IO | Changes I/O parrallelism by tablespace. Often set to * . If you have high prefetch wait times, the application may be waiting on pages and this may need to be adjusted.|
| DB2COMM | Setting communication protocols. This is almost always TCPIP. |
| DB2_USE_ALTERNATE_PAGE_CLEANING | Tells Db2 to be more proactive with page cleaning. Usually set to ON in transaction processing databases. |
| DB2AUTH | Change authentication behavior. When LDAP is enabled, this should be OSAUTHDB. | 
|DB2_RESTORE_GRANT_ADMIN_AUTHORITIES | Usually set to ON. Immediately grants instance ownser SECADM which allows database restore from a backup taken in another instance (i.e. another server). |

In [ ]:
%%sql mon_settings <<
SELECT 
  NAME AS MONITOR_VALUE,
  VALUE AS IS_ON
FROM SYSIBMADM.DBMCFG
WHERE NAME IN ('dft_mon_bufpool','dft_mon_lock','dft_mon_sort','dft_mon_stmt','dft_mon_table','dft_mon_timestamp','dft_mon_uow')

In [ ]:
display(Markdown("#### Are Database Metrics Being Collected (DBM CFG)?"))
display(Markdown("My personal best practice is for all DFT_MON values should be set to ON."))
mon_settings_df=mon_settings.DataFrame()
mon_settings_df.columns=mon_settings_df.columns.str.upper()
mon_settings_df

In [ ]:
%%sql key_dbm_config <<
SELECT 
  NAME AS DBM_VALUE,
  VALUE AS CURRENT_SETTING,
  VALUE_FLAGS AS FLAGS
FROM SYSIBMADM.DBMCFG
WHERE NAME IN ('svcename','diagpath','diaglevel','instance_memory','intra_parallel','keystore_type','keystore_location')
ORDER BY NAME 
WITH UR

In [ ]:
display(Markdown("#### Key Database Manager Settings (DBM CFG)"))
key_dbm_config_df=key_dbm_config.DataFrame()
key_dbm_config_df.columns=key_dbm_config_df.columns.str.upper()
key_dbm_config_df

| Parameter   | Note |
| :----------- | :----------- |
| diaglevel | Level of diagnostic messages captured in Db2 error log. Usually set to 3. All errors, warnings, event messages, and administration notification messages are captured. Some key informational messages might be captured at this level as well. |
| diagpath | Where the Db2 error log is held (db2diag.log) |
| instance_memory | Maximum amount of memory at instance level. When set to AUTOMATIC this is calculated between 75-90% of system memory. <b>Warning:</b> <i>This should be set to a cap on a containerized database. Otherwise Db2 will use node memory to calculate it's setting. This may starve out other workloads.</i> | 
| intra_parallel | Affects intrapartition query paralellism. Very dangerous to have on a OLTP database without heavy testing. Should be set to NO. |
| keystore_location | Location of encryption keys. |
| keystore_type | Type of keystore used to store encryption keys.
| svcename | Name of the port listed in /etc/services. |

---

## Database Configuration & General Performance

<font color=blue>
In this section we are looking at how a database is configured and how it generally behaves.<br>
    
We are looking for details like how long the database has been up and running and what best practices may or may not be in place. It also allows us to look at general expected behavior and efficiency at a very high level. 
</font>

In [ ]:
%%sql database_uptime <<
SELECT
VARCHAR_FORMAT(DB_CONN_TIME, 'YYYY-MM-DD HH24:MM:SS') AS DATABASE_ACTIVATION
    FROM TABLE(MON_GET_DATABASE(-2))
WITH UR

In [ ]:
display(Markdown("#### Database Start time"))
display(Markdown("Note: Many notebook calculations use cumulative database metrics collected from database start time. You could be looking at 2 days or 2 years of information. Metrics will change on database restart, which will reset the cumulative data."))
database_uptime_df=database_uptime.DataFrame()
database_uptime_df.columns=database_uptime_df.columns.str.upper()
database_uptime_df

In [ ]:
%%sql db_last_bkup <<
SELECT VARCHAR_FORMAT(LAST_BACKUP, 'YYYY-MM-DD HH24:MM:SS') AS LAST_DATABASE_BACKUP
    FROM TABLE(MON_GET_DATABASE(-2))
WITH UR;

In [ ]:
display(Markdown("#### Last Database Backup"))
display(Markdown("Are database backups being routinely taken?"))
db_last_bkup_df=db_last_bkup.DataFrame()
db_last_bkup_df.columns=db_last_bkup_df.columns.str.upper()
db_last_bkup_df

In [ ]:
%%sql colocated <<
SELECT
 CASE
  WHEN TYPE = 'DB_STORAGE_PATH' THEN 'AUTOMATIC STORAGE PATH'
  WHEN TYPE = 'LOGPATH' THEN 'ACTIVE LOG DIRECTORY PATH'
  WHEN TYPE = 'DBPATH' THEN 'DATABASE DIRECTORY PATH'
  ELSE TYPE
 END AS TYPE,
 SUBSTR(PATH,1,60) AS PATH
 FROM SYSIBMADM.DBPATHS
 WHERE TYPE NOT IN ('LOCAL_DB_DIRECTORY')
UNION ALL
 SELECT
  CASE
   WHEN NAME = 'logarchmeth1' THEN 'ARCHIVE LOG DIRECTORY PATH'
   ELSE NAME
  END AS TYPE,
  REPLACE(VALUE,'DISK:','') AS PATH
 FROM SYSIBMADM.DBCFG
 WHERE NAME IN ('logarchmeth1') AND VALUE LIKE 'DISK:%'
UNION ALL
SELECT
  CASE
   WHEN NAME = 'diagpath' THEN 'DB2 ERROR LOG PATH'
   ELSE NAME
  END AS TYPE,
  VALUE AS PATH
 FROM SYSIBMADM.DBMCFG
 WHERE NAME IN ('diagpath')
WITH UR

In [ ]:
display(Markdown("#### Colocated Filesystem Check"))
display(Markdown("Typically backend filesystems are seperated for active logging, archive logging, backup, error log, and data. This is muddied in a container or cloud based installation, but this practice should be respected whenever possible. It prevents I/O contention and one process choking out the space requirements of another process."))
db_colocated_df=colocated.DataFrame()
db_colocated_df.columns=db_colocated_df.columns.str.upper()
db_colocated_df

In [ ]:
%%sql key_db_cfg << 
SELECT 
  NAME AS DB_VALUE,
  VALUE AS CURRENT_SETTING,
  VALUE_FLAGS AS FLAGS
FROM SYSIBMADM.DBCFG 
WHERE NAME IN ('database_memory','locktimeout','logarchmeth1','logfilsiz','logprimary','logsecond','numlogspan', 'num_iocleaners','num_ioservers','logarchcompr1','self_tuning_mem')
ORDER BY NAME
WITH UR

In [ ]:
display(Markdown("#### Key Database Configuration Settings (DB CFG)"))
display(Markdown("There are MANY more configuration settings, this selection narrows our focus to a few key areas."))
key_db_cfg_df=key_db_cfg.DataFrame()
key_db_cfg_df.columns=key_db_cfg_df.columns.str.upper()
key_db_cfg_df

| Parameter   | Note |
| :----------- | :----------- |
| database_memory | Amount of memory allocated to the database (subset of instance memory). AUTOMATIC allows Db2 to make decision on size it thinks is best based on current architecture and design. |
| locktimeout | Number of seconds an application will wait to obtain a lock. If exceeded, the SQL times out. Usually set to 60 seconds. |
| logarchcompr1 | Specifies if archive logs are compressed (saving space in the filesystem). |
| logarchmeth1 | Specifies Db2 should use archive logging for point in time recovery and specifies where the archive logs should be held. May be local on server or in a S3 bucket. |
| logfilsiz | Size of active logs. Often dictated by architecture and then tuned by DBA. |
| logprimary | Number of primary active logs used by Db2. Varies by installation and prerequisites. |
| logsecond | Number of secondary active logs used by Db2 when under load. Varies by installation and prerequisites. |
| num_iocleaners | Number of asyncronous page cleaners used by the database. Rule of thumb is a cleaner for each physical CPU. When set to AUTOMATIC, it is set to a cleaner per physical core. |
| num_ioservers | Number of prefetchers that can be used in a database at any one time for prefecthing and asyncronous I/O. AUTOMATIC sets this to parallellism settings of the tablespaces. |
| self_tuning_mem | Tells Db2 to dynamically manage available memory for consumers enabled for self tuning. Typically this is always set to ON. |

In [ ]:
%%sql hadr_state <<
SELECT 
	HADR_CONNECT_STATUS,								
	HADR_STATE,
	HADR_LOG_GAP
from table (MON_GET_HADR(0))
WITH UR

In [ ]:
display(Markdown("#### HADR Connection State"))
display(Markdown("If HADR is configured, this detail will show us if it is running and communicating normally."))
hadr_state_df=hadr_state.DataFrame()
hadr_state_df.columns=hadr_state_df.columns.str.upper()
hadr_state_df

In [ ]:
%%sql hadr_config <<
SELECT
	NAME AS DB_VALUE,
	VALUE AS CURRENT_SETTING
FROM SYSIBMADM.DBCFG
WHERE NAME IN ('hadr_local_host','hadr_local_svc','hadr_remote_host','hadr_remote_svc','hadr_syncmode','hadr_timeout','hadr_peer_window','hadr_target_list','hadr_replay_delay','logindexbuild','blocknonlogged','log_ddl_stmts')
WITH UR

In [ ]:
display(Markdown("#### HADR Database Configuration Settings (DB CFG)"))
display(Markdown("These parameters give us an idea of how high availability and disaster recovery (HADR) is configured as well as the settings for a few best practices."))
hadr_config_df=hadr_config.DataFrame()
hadr_config_df.columns=hadr_config_df.columns.str.upper()
hadr_config_df

| Parameter   | Note |
| :----------- | :----------- |
| blocknonlogged | Will prevent non-logged operations which helps ensure transaction logging is not bypassed so data is replicated to standby as normal. |
| hadr_local_host | Hostname of the primary database server.|
| hadr_local_svc | Port number HADR is listening on for the primary server. |
| hadr_remote_host | Hostname of the standby database server.|
| hadr_remote_svc | Port number HADR is listening on for the standby server. |
| hadr_syncmode | Determines how log writes on the primary server are synced with log writes on the standby server. |
| hadr_timeout | Number of seconds HADR process waits before considering a communication attempt to have failed. Usually set between 60 and 120.|
| hadr_peer_window | Number of seconds the primary-standby pair continues to behave in peer state if primary loses connection to standby. Usually set between 60 and 120. |
| logindexbuild | Index creation/recreation is logged on primary so they can be reconstructed on standby. Should be set to ON. |
| log_ddl_stmts | DDL operations are recorded in logs on primary to provide more detail to the standby. Should be set to YES. |
| hadr_target_list | Failover priority list when a failover takes place. Required when you have more than one standby. General best practice if you only have a single standby. |
| hadr_replay_delay | Number of seconds from transaction commit on primary ro transaction commit on standby. Sometimes used to delay HADR state in case time is needed to disconnect the two because a bad transaction was about to be applied. Should be set to 0 for no delay. |



In [ ]:
%%sql db_arss <<
SELECT 
	ROWS_RETURNED,
	SELECT_SQL_STMTS,
	CASE 
		WHEN SELECT_SQL_STMTS > 0 
			THEN DECIMAL(FLOAT(rows_returned)/FLOAT(SELECT_SQL_STMTS),10,2)
			ELSE 'No Div. By zero'
	END AS AVG_RESULT_SET_SIZE,
	CASE 
		WHEN DECIMAL(FLOAT(rows_returned)/FLOAT(SELECT_SQL_STMTS),10,2) <= 10 
			THEN 'Transaction Processing Like Workload' 
			ELSE 'Warehouse Like Workload'
	END AS WORKLOAD_TYPE
FROM TABLE(MON_GET_DATABASE(-2))
WITH UR

In [ ]:
display(Markdown("#### Database Average Result Set Size (ARSS)"))
display(Markdown("Does our database workload match what we expect it to be? Do we expect a very fast transactional workload but show a result set size typical of a warehouse or vise versa?"))
db_arss_df=db_arss.DataFrame()
db_arss_df.columns=db_arss_df.columns.str.upper()
db_arss_df

| Avg Result Set Size | Typical Workload Type |
| :----------- | :----------- |
| Less than 10 | Transaction Processing Workload |
| More than 10 | Mixed Workload or Data Warehouse Workload |

In [ ]:
%%sql db_ixref <<
SELECT ROWS_READ, 
 ROWS_RETURNED,
 ROWS_READ/ROWS_RETURNED AS IXREF 
FROM TABLE(MON_GET_WORKLOAD('',-2)) AS T 
WHERE WORKLOAD_NAME='SYSDEFAULTUSERWORKLOAD' 
WITH UR

In [ ]:
display(Markdown("#### Database Index Read Efficiency (IXREF)"))
display(Markdown("On average, for the database, how many rows are scanned before Db2 finds the row it needs? Often shows effectiveness of indexes and how they are used."))
db_ixref_df=db_ixref.DataFrame()
db_ixref_df.columns=db_ixref_df.columns.str.upper()
db_ixref_df

| Index Read Efficiency | Efficiency Level |
| :----------- | :----------- |
| Less than 10 | Ideal (Especially for Transaction Processing) |
| 10 to 100| Potentially acceptable. Possibly a mixed workload. |
| 100 to 1000 | Poor. There are SQL tuning opportunities here. |
| Greater than 1000 | Bad. Many SQL and DB tuning opportunities. |

In [ ]:
%%sql db_rollbackpercent <<
SELECT TOTAL_APP_COMMITS AS APP_COMMITS, 
	   TOTAL_APP_ROLLBACKS AS APP_ROLLBACKS,
	   TOTAL_APP_COMMITS + TOTAL_APP_ROLLBACKS AS TOTAL,
	   CASE
            WHEN TOTAL_APP_COMMITS + TOTAL_APP_ROLLBACKS > 0 THEN
                DECIMAL(FLOAT(TOTAL_APP_ROLLBACKS)/FLOAT(TOTAL_APP_COMMITS + TOTAL_APP_ROLLBACKS),10,2)*100
            ELSE 0
        END AS PCNT_ROLLBACK
FROM TABLE(MON_GET_DATABASE(-2))
WITH UR

In [ ]:
display(Markdown("#### Database Transaction Rollback Percentage"))
display(Markdown("What percentage of your total workload is wasted on rolling back work that was in-flight? Anything over 1% would be uncommon. Excessive rollbacks can lead to locking and contention in database."))
db_rollbackpercent_df=db_rollbackpercent.DataFrame()
db_rollbackpercent_df.columns=db_rollbackpercent_df.columns.str.upper()
db_rollbackpercent_df

In [ ]:
db_rollback= %sql SELECT TOTAL_APP_COMMITS AS APP_COMMITS, \
	TOTAL_APP_ROLLBACKS AS APP_ROLLBACKS \
FROM TABLE(MON_GET_DATABASE(-2)) \
WITH UR

db_rollback_df=db_rollback.DataFrame()
db_rollback_df.sum().plot(kind='pie', autopct='%1.0f%%', ylabel=" ", title='Percent of Database Rollbacks')


In [ ]:
%%sql bp_ratio <<
-- LISTAGG CHR10 inerts a newline which can not be displayed as a dataframe
-- As a result we can not uppercase the column headers and use only the display command

SELECT
	SUBSTR(BPHR.BP_NAME,1,18) AS BP_NAME,
	BP_CUR_BUFFSZ
	PAGESIZE,
	((PAGESIZE*BP_CUR_BUFFSZ)/1024)/1024 AS SZ_MB,
	TOTAL_LOGICAL_READS,
	TOTAL_PHYSICAL_READS,
	DATA_HIT_RATIO_PERCENT,
	(SELECT LISTAGG(TBSPACE,CHR(10)) 
		WITHIN GROUP (ORDER BY CREATE_TIME) 
		FROM SYSCAT.TABLESPACES TS, SYSCAT.BUFFERPOOLS B 
		WHERE TS.BUFFERPOOLID = B.BUFFERPOOLID AND B.BPNAME=MGBP.BP_NAME) AS TABLESPACES
FROM SYSIBMADM.BP_HITRATIO BPHR JOIN TABLE(MON_GET_BUFFERPOOL(NULL,-2)) MGBP
	ON MGBP.BP_NAME=BPHR.BP_NAME
    JOIN SYSCAT.BUFFERPOOLS SBP ON SBP.BPNAME=MGBP.BP_NAME
WITH UR

In [ ]:
display(Markdown("#### Bufferpool Ratios and Tablespace Mapping"))
display(Markdown("The focus here is the Bufferpool Hit Ratio. Ideally you want to be 99-98%. Lower than 90 and there is probably opportunity for tuning."))
display(bp_ratio)

In [ ]:
%%sql bp_tbl_cnt <<
SELECT 
	B.BPNAME AS BUFFERPOOL,
	B.BUFFERPOOLID AS BP_ID,
	(SELECT COUNT(*) 
		FROM SYSCAT.TABLESPACES TS 
		WHERE B.BUFFERPOOLID=TS.BUFFERPOOLID) AS TBLSPC_COUNT,
	(SELECT COUNT(*)
       FROM SYSCAT.TABLES T JOIN SYSCAT.TABLESPACES TS
	   ON T.TBSPACE=TS.TBSPACE OR T.INDEX_TBSPACE=TS.TBSPACE OR T.LONG_TBSPACE=TS.TBSPACE
       WHERE TS.BUFFERPOOLID=B.BUFFERPOOLID) AS TABLE_COUNT
    FROM SYSCAT.BUFFERPOOLS B
WITH UR

In [ ]:
display(Markdown("#### Mapping: Bufferpool to Tablespace and Table Count"))
display(Markdown("Often tablespaces and tables are grouped into asyncronous or synronus workloads and broken up into thier own tablespace or bufferpool. Critical or large tables may also be split off. This data shows general distrobution to give perspective of skew and mapping."))
bp_tbl_cnt_df=bp_tbl_cnt.DataFrame()
bp_tbl_cnt_df.columns=bp_tbl_cnt_df.columns.str.upper()
bp_tbl_cnt_df

In [ ]:
%%sql db_invalid <<
SELECT
	CASE OBJECTTYPE
	 WHEN 'B' THEN 'TRIGGER'
	 WHEN 'F' THEN 'ROUTINE'
	 WHEN 'R' THEN 'USER-DEFINED DATA TYPE'
	 WHEN 'V' THEN 'VIEW'
	 WHEN 'v' THEN 'GLOBAL VARIABLE'
	 WHEN 'y' THEN 'ROW PERMISSION'
	 WHEN '2' THEN 'COLUMN MASK'
	 WHEN '3' THEN 'USAGE LIST'
	END
	OBJECT_TYPE,
	OBJECTSCHEMA AS	SCHEMA ,
	ROUTINENAME AS ROUTINE_NAME,
	OBJECTNAME AS OBJECT_NAME,
	SQLCODE,
	DATE(INVALIDATE_TIME)	AS INVALID_DATE,
	TIME(INVALIDATE_TIME)	AS INVALID_TIME,
	DATE(LAST_REGEN_TIME)	AS LAST_REGEN
FROM SYSCAT.INVALIDOBJECTS
WHERE OBJECTSCHEMA = :schema
ORDER BY OBJECT_TYPE, OBJECTSCHEMA
WITH UR

In [ ]:
display(Markdown("#### Invalid Database Objects"))
display(Markdown("Are there any database objects that are marked invalid and need to be rebuilt or revalidated?"))
db_invalid_df=db_invalid.DataFrame()
db_invalid_df.columns=db_invalid_df.columns.str.upper()
db_invalid_df

---

## Table Performance

<font color=blue>
In this section we are looking at how efficiently SQL runs through the database by analyzing table behavior.<br>

We can analyze general performance by looking at hot tables, what happens when a table is accessed, and how efficient an index may be. If database efficiency metrics in the previous section were not what we wanted or expected, this is a more granular look at why.    
</font>

In [ ]:
%%sql table_byreadoverflow << 
WITH t AS (
 SELECT
      TABSCHEMA AS SCHEMA,
      TABNAME AS TABLE,
      ROWS_READ,
      OVERFLOW_ACCESSES,
      CASE WHEN ROWS_READ > 0 
        THEN DECIMAL(FLOAT(OVERFLOW_ACCESSES)/FLOAT(ROWS_READ),10,2)*100 
        ELSE 0 
      END AS R_OVRFLW_PCT
 FROM TABLE(MON_GET_TABLE('','',-2)) AS t WHERE TABSCHEMA =:schema AND OVERFLOW_ACCESSES > 0)
SELECT * FROM t WHERE R_OVRFLW_PCT >= 3 ORDER BY R_OVRFLW_PCT DESC
WITH UR

In [ ]:
display(Markdown("#### Tables > 3% Read Overflows"))
display(Markdown("A read overflow is the result of Db2 looking for data where it expects to be and finds a pointer redirecting it to a new location that actually has the data. This causes a double logical read (essentially twice the work normally needed). It's a good practice to run a Db2 REORG on tables with more than 3% Read Overflows. This updates Db2's knowledge of where the data actually is."))
table_byreadoverflow_df=table_byreadoverflow.DataFrame()
table_byreadoverflow_df.columns=table_byreadoverflow_df.columns.str.upper()
table_byreadoverflow_df

In [ ]:
%%sql table_bywriteoverflow << 
WITH t AS (
 SELECT
      TABSCHEMA AS SCHEMA,
      TABNAME AS TABLE,
      ROWS_UPDATED,
      OVERFLOW_CREATES,
      CASE WHEN ROWS_UPDATED > 0 
        THEN DECIMAL(FLOAT(OVERFLOW_CREATES)/FLOAT(ROWS_UPDATED),10,2)*100 
        ELSE 0 
      END AS W_OVRFLW_PCT
 FROM TABLE(MON_GET_TABLE('','',-2)) AS t WHERE TABSCHEMA =:schema AND OVERFLOW_CREATES > 0)
SELECT * FROM t WHERE W_OVRFLW_PCT >= 3 ORDER BY W_OVRFLW_PCT DESC
WITH UR

In [ ]:
display(Markdown("#### Tables > 3% Write Overflows"))
display(Markdown("Similiar to the Read Overflow, but this occurs when writing data. An update operation to VARCHAR causes the field to be longer and not fit on the original planned data page. A pointer is left and the data is written on another page. It's a good practice to run a Db2 REORG on tables with more than 3% Write Overflows."))
table_bywriteoverflow_df=table_bywriteoverflow.DataFrame()
table_bywriteoverflow_df.columns=table_bywriteoverflow_df.columns.str.upper()
table_bywriteoverflow_df

In [ ]:
%%sql table_top10_byinsert << 
SELECT * 
FROM (SELECT ROW_NUMBER() OVER(ORDER BY SUM(ROWS_INSERTED) DESC) AS RANK, 
       TRIM(TABSCHEMA) as SCHEMA, 
       TRIM(TABNAME) as TABLE, 
       SUM(ROWS_INSERTED) as ROWS_INSERTED
      FROM TABLE(MON_GET_TABLE('','',-2)) AS TB
      WHERE TABSCHEMA=:schema
      GROUP BY TABSCHEMA, TABNAME) 
WHERE RANK<=10
WITH UR

In [ ]:
display(Markdown("#### Top 10 Tables by Rows Inserted"))
table_top10_byinsert_df=table_top10_byinsert.DataFrame()
table_top10_byinsert_df.columns=table_top10_byinsert_df.columns.str.upper()
table_top10_byinsert_df

In [ ]:
%%sql table_top10_byread << 
SELECT * 
FROM (SELECT ROW_NUMBER() OVER(ORDER BY SUM(ROWS_READ) DESC) AS RANK, 
       TRIM(TABSCHEMA) AS SCHEMA, 
       TRIM(TABNAME) AS TABLE, 
       SUM(ROWS_READ) AS ROWS_READ
      FROM TABLE(MON_GET_TABLE('','',-2)) AS TB 
      WHERE TABSCHEMA=:schema
      GROUP BY TABSCHEMA, TABNAME) 
WHERE RANK<=10
WITH UR

In [ ]:
display(Markdown("#### Top 10 Tables by Rows Read"))
table_top10_byread_df=table_top10_byread.DataFrame()
table_top10_byread_df.columns=table_top10_byread_df.columns.str.upper()
table_top10_byread_df

In [ ]:
%%sql table_top10_byupdate <<
SELECT * 
FROM (SELECT ROW_NUMBER() OVER(ORDER BY SUM(ROWS_UPDATED) DESC) AS RANK, 
       TRIM(TABSCHEMA) AS SCHEMA, 
       TRIM(TABNAME) AS TABNAME, 
       SUM(ROWS_UPDATED) AS ROWS_UPDATED
      FROM TABLE(MON_GET_TABLE('','',-2)) AS TB 
      WHERE TABSCHEMA=:schema
      GROUP BY TABSCHEMA, TABNAME) 
WHERE RANK<=10
WITH UR

In [ ]:
display(Markdown("#### Top 10 Tables by Rows Updated"))
table_top10_byupdate_df=table_top10_byupdate.DataFrame()
table_top10_byupdate_df.columns=table_top10_byupdate_df.columns.str.upper()
table_top10_byupdate_df

In [ ]:
%%sql table_top10_bydelete <<
SELECT * 
FROM (SELECT ROW_NUMBER() OVER(ORDER BY SUM(ROWS_DELETED) DESC) AS RANK,
       TRIM(TABSCHEMA) AS SCHEMA, 
       TRIM(TABNAME) AS NAME, 
       SUM(ROWS_DELETED) AS ROWS_DELETED
      FROM TABLE(MON_GET_TABLE('','',-2)) AS TB
      WHERE TABSCHEMA=:schema
      GROUP BY TABSCHEMA, TABNAME) 
WHERE RANK<=10
WITH UR

In [ ]:
display(Markdown("#### Top 10 Tables by Rows Deleted"))
table_top10_bydelete_df=table_top10_bydelete.DataFrame()
table_top10_bydelete_df.columns=table_top10_bydelete_df.columns.str.upper()
table_top10_bydelete_df

In [ ]:
%%sql table_top10_byscan <<
SELECT * 
FROM (SELECT ROW_NUMBER() OVER(ORDER BY SUM(TABLE_SCANS) DESC) AS RANK, 
       TRIM(TABSCHEMA) AS SCHEMA, 
       TRIM(TABNAME) AS TABLE, 
       SUM(TABLE_SCANS) AS NUM_SCANS
      FROM TABLE(MON_GET_TABLE('','',-2)) AS TB 
      GROUP BY TABSCHEMA, TABNAME) 
WHERE RANK<=10
WITH UR

In [ ]:
display(Markdown("#### Top 10 Tables by Number of Scans"))
table_top10_byscan_df=table_top10_byscan.DataFrame()
table_top10_byscan_df.columns=table_top10_byscan_df.columns.str.upper()
table_top10_byscan_df

In [ ]:
%%sql top_lowcard_indexes <<

-- Find top 25 busiest tables on 100 rows or more
-- Calculate index cardinality percentage compared to total table cardinality
-- Show index cardinality percentage less than to equal to 3%
-- Do not take (U)nique indexes into account because they are used to enforce uniqueness

SELECT A.TABSCHEMA AS SCHEMA,
       A.TABNAME AS TABLE,
       A.INDNAME AS INDEX_NAME,
       B.CARD AS TB_CARD,
       A.FULLKEYCARD AS IX_FULLKEYCARD,
       INT((FLOAT(A.FULLKEYCARD)/FLOAT(B.CARD)) * 100) AS PCNT_TBCARD
FROM SYSCAT.INDEXES A INNER JOIN SYSCAT.TABLES B
       ON A.TABSCHEMA = B.TABSCHEMA
       AND A.TABNAME = B.TABNAME
WHERE A.FULLKEYCARD > 0
     AND A.TABSCHEMA <> 'SYSIBM'
 	 AND B.CARD > 100
 	 AND A.UNIQUERULE <> 'U'
 	 AND INT((FLOAT(A.FULLKEYCARD)/FLOAT(B.CARD)) * 100) <= 3
 	 AND A.TABSCHEMA=:schema
 	 AND A.TABNAME IN 
	   (SELECT TABNAME
       FROM TABLE(MON_GET_TABLE('','',-2)) 
       WHERE TABSCHEMA=:schema
       ORDER BY ROWS_INSERTED + ROWS_UPDATED + ROWS_DELETED DESC
       FETCH FIRST 25 ROWS ONLY)
ORDER BY INT((FLOAT(A.FULLKEYCARD)/FLOAT(B.CARD)) * 100) ASC, B.CARD DESC
WITH UR

In [ ]:
display(Markdown("#### Low Cardinality Indexes (of Top 25 Busiest Tables)"))
display(Markdown("The higher the FULLKEYCARD is (in relation to table cardinality) the more unique the value is. This leads to an efficient index. The lower FULLKEYCARD value means less unique values (1 being all values the same) which causes an inefficient index. This provides a starting point of what tables may need existing indexes tuned."))
top_lowcard_indexes_df=top_lowcard_indexes.DataFrame()
top_lowcard_indexes_df.columns=top_lowcard_indexes_df.columns.str.upper()
top_lowcard_indexes_df

In [ ]:
%%sql top_tables_unused_indexes <<

-- Make sure to exclude Primary Keys
-- Watch out for Cluster Keys as MDC's don't seem to update LAST_USED

SELECT
	TABSCHEMA,
	TABNAME,
	COUNT(INDNAME) AS IDX_NOT_USED_CNT
FROM SYSCAT.INDEXES
WHERE LASTUSED='0001-01-01' AND tabschema=:schema AND UNIQUERULE != 'P'
AND TABNAME IN
 	   (SELECT TABNAME
       FROM TABLE(MON_GET_TABLE('','',-2))
       WHERE TABSCHEMA=:schema
       ORDER BY ROWS_INSERTED + ROWS_UPDATED + ROWS_DELETED DESC
       FETCH FIRST 25 ROWS ONLY)
GROUP BY TABSCHEMA,TABNAME
ORDER BY IDX_NOT_USED_CNT DESC
WITH UR

In [ ]:
display(Markdown("#### Number of Indexes That Have Never Been Used (of Top 25 Busiest Tables)"))
display(Markdown("The more indexes a table has, the more of a performance impact on INSERT operations. These tables have high write activity but have indexes that are never used and could be slowing down INSERT performance."))
top_tables_unused_indexes_df=top_tables_unused_indexes.DataFrame()
top_tables_unused_indexes_df.columns=top_tables_unused_indexes_df.columns.str.upper()
top_tables_unused_indexes_df

---

## Index Performance

<font color=blue>
In this section we are looking at how efficiently SQL runs through the database by analyzing Index Read Efficiency (IXREF).<br>

We looked at this from a database perspective in the Database Configuration & General Performance section. The detail below will show what inefficient SQL drove that number. These SQL are scanning many rows to get to the row it is seeking.    
</font>

In [ ]:
%%sql stmt_top10_byixref <<
WITH SUM_TAB (SUM_RR) AS (
        SELECT FLOAT(SUM(ROWS_READ))
        FROM TABLE(MON_GET_PKG_CACHE_STMT ( 'D', NULL, NULL, -2)) AS T)
SELECT
        INSERT_TIMESTAMP AS FIRST_INSRT_INTO_CACHE,
        SUBSTR(STMT_TEXT,1,50) AS STATEMENT,
        ROWS_READ,
        DECIMAL(100*(FLOAT(ROWS_READ)/SUM_TAB.SUM_RR),5,2) AS PCT_TOT_RR,
        ROWS_RETURNED,
        CASE
            WHEN ROWS_RETURNED > 0 THEN
                DECIMAL(FLOAT(ROWS_READ)/FLOAT(ROWS_RETURNED),10,2)
            ELSE -1
        END AS IXREF,
        NUM_EXECUTIONS
    FROM TABLE(MON_GET_PKG_CACHE_STMT ( 'D', NULL, NULL, -2)) AS T, SUM_TAB
    ORDER BY ROWS_READ DESC FETCH FIRST 10 ROWS ONLY 
    WITH UR

In [ ]:
display(Markdown("#### Top 10 Inefficient Statements (IXREF)"))
display(Markdown("Index Read Efficiency shows how many rows are scanned to fetch the row DB2 needs. Ideally in a OLTP environment this should be <10. High IXREF SQL that executes frequently could be a performance problem. Keep in mind, the timestamp of first insert into package cache can change on rerun as things are flushed out, etc. The data you are seeing are details since that first insert into the cache."))
stmt_top10_byixref_df=stmt_top10_byixref.DataFrame()
stmt_top10_byixref_df.columns=stmt_top10_byixref_df.columns.str.upper()
stmt_top10_byixref_df

---

## Database Wait Time

<font color=blue>
In this section we are looking at where our database spends time waiting as it does work. Waiting isn't necessarily bad if it isn't excessive and is in a more efficient area (ie. reading memory) than a less efficient area (ie. spinning physical disk).
</font>

In [ ]:
%%sql db_timespent << select * 
    from table(mon_get_database(-2));

In [ ]:
df=db_timespent.DataFrame()
df.columns= df.columns.str.upper()
df.columns

In [ ]:
# Input: 'df' is a DataFrame that has time series data from MON_GET_DATABASE.
# Because the Wait Times are cumulative, I really only care about the most 
# recent entry in 'df'.
#
# Get the index of the last row, and build a dictionary with with all of the 
# wait time elements:
i = 0
ix = {'AGENT_WAIT_TIME': df['AGENT_WAIT_TIME'][i],
      'WLM_QUEUE_TIME_TOTAL': df['WLM_QUEUE_TIME_TOTAL'][i],
      'LOCK_WAIT_TIME':  df['LOCK_WAIT_TIME'][i], 
      'LOG_BUFFER_WAIT_TIME': df['LOG_BUFFER_WAIT_TIME'][i],
      'LOG_DISK_WAIT_TIME': df['LOG_DISK_WAIT_TIME'][i],
      'TCPIP_RECV_WAIT_TIME': df['TCPIP_RECV_WAIT_TIME'][i],
      'TCPIP_SEND_WAIT_TIME': df['TCPIP_SEND_WAIT_TIME'][i],
      'IPC_RECV_WAIT_TIME': df['IPC_RECV_WAIT_TIME'][i],
      'IPC_SEND_WAIT_TIME': df['IPC_SEND_WAIT_TIME'][i],
      'FCM_RECV_WAIT_TIME': df['FCM_RECV_WAIT_TIME'][i],
      'FCM_SEND_WAIT_TIME': df['FCM_SEND_WAIT_TIME'][i],
      'AUDIT_SUBSYSTEM_WAIT_TIME': df['AUDIT_SUBSYSTEM_WAIT_TIME'][i],
      'AUDIT_FILE_WRITE_WAIT_TIME': df['AUDIT_FILE_WRITE_WAIT_TIME'][i],
      'DIAGLOG_WRITE_WAIT_TIME': df['DIAGLOG_WRITE_WAIT_TIME'][i],
      'POOL_READ_TIME': df['POOL_READ_TIME'][i],
      'POOL_WRITE_TIME': df['POOL_WRITE_TIME'][i],
      'DIRECT_READ_TIME': df['DIRECT_READ_TIME'][i],
      'DIRECT_WRITE_TIME': df['DIRECT_WRITE_TIME'][i],
      'EVMON_WAIT_TIME': df['EVMON_WAIT_TIME'][i],
      'TOTAL_EXTENDED_LATCH_WAIT_TIME': df['TOTAL_EXTENDED_LATCH_WAIT_TIME'][i],
      'PREFETCH_WAIT_TIME': df['PREFETCH_WAIT_TIME'][i],
      'COMM_EXIT_WAIT_TIME': df['COMM_EXIT_WAIT_TIME'][i],
      'IDA_SEND_WAIT_TIME': df['IDA_SEND_WAIT_TIME'][i],
      'IDA_RECV_WAIT_TIME': df['IDA_RECV_WAIT_TIME'][i],
      'CF_WAIT_TIME': df['CF_WAIT_TIME'][i],
      'RECLAIM_WAIT_TIME': df['RECLAIM_WAIT_TIME'][i],
      'SPACEMAPPAGE_RECLAIM_WAIT_TIME': df['SPACEMAPPAGE_RECLAIM_WAIT_TIME'][i]}

# Convert the dictionary into a new DataFrame.
myx=pd.DataFrame.from_dict(ix, orient='index')

# Calcuate the percentage of overall wait for each value.
total=myx[0].sum()
myx['PCT'] = myx.apply(lambda x: x[0]/total, axis=1)

# Filter out wait elements that have <= 0.5% wait time), to avoid having 
# 27 slices of pie, most of which are tiny slivers
myx = myx[myx['PCT'] >= 0.005]

# Set up the graph
display(Markdown("#### Database Wait Time"))
display(Markdown("Where is your database spending most of it's time waiting when doing work?"))
xlabel=myx.index.tolist()
yname=myx.columns.values[0]
plt = myx.plot(kind='pie',y=yname,labels=xlabel,figsize=[5,5],autopct='%0.1f%%')
plt.get_legend().remove()
plt.axes.get_yaxis().set_visible(False)

---

## Locking Analysis (Past 24 Hours)

<font color=blue>
Locking is a normal behavior in a database. However, cerian types of locking phenomena be a early warning sign for contention problems. This section will take a look at locking behavior over a 24 hour period.
</font>

In [ ]:
%%sql lock_event_24 <<
SELECT
  SUBSTR(EVENT_TYPE,1,18) AS EVENT_TYPE,
  COUNT(*)/2 AS COUNT
FROM DBAMON.LOCK_EVENT
WHERE EVENT_TIMESTAMP >= CURRENT TIMESTAMP - 24 hours
GROUP BY EVENT_TYPE 
WITH UR

In [ ]:
display(Markdown("####  Locking Events - Past 24 hours"))
display(Markdown("Locking is a natural part of work. The question is if any contention,timeouts, or waiting are excessive. Personal rule of thumb is anything under 10 occurances (that are not clustered in a short timeframe) over a 24 hour period is perfectly fine."))
lock_event_24_df=lock_event_24.DataFrame()
lock_event_24_df.columns=lock_event_24_df.columns.str.upper()
lock_event_24_df

| Event Type| Count |
| :----------- | :----------- |
| Deadlock | Two applications each hold a lock on a object the other needs. Db2 will choose a winner and a loser. The loser will lose it's lock so the other can proceed. Usually sign of an application problem. |
| Locktimeout | One transaction is waiting for the lock on an object to be released and aborts when a specific time limit is reached. |
| Lockwait | When a transaction is waiting for a lock on an object to be released. May eventually lead to a LOCKTIMEOUT if that pre-determined wait time is exceeded. |

In [ ]:
%%sql lock_event_bytable_24 <<
SELECT
  SUBSTR(LP.TABLE_SCHEMA,1,18) AS TABLE_SCHEMA,
  SUBSTR(LP.TABLE_NAME,1,30) AS TABLE_NAME,
  SUBSTR(LE.EVENT_TYPE,1,18) AS LOCK_EVENT,
  COUNT(*)/2 AS COUNT
FROM DBAMON.LOCK_PARTICIPANTS LP, DBAMON.LOCK_EVENT LE
WHERE LP.XMLID=LE.XMLID 
	AND EVENT_TIMESTAMP >= CURRENT TIMESTAMP - 24 HOURS
	AND NOT (LP.TABLE_SCHEMA IS NULL OR LP.TABLE_SCHEMA IS NULL)
GROUP BY LP.TABLE_SCHEMA, LP.TABLE_NAME, LE.EVENT_TYPE
ORDER BY LP.TABLE_SCHEMA, LP.TABLE_NAME, LE.EVENT_TYPE 
WITH UR

In [ ]:
display(Markdown("####  Locking Events By Table - Past 24 hours"))
display(Markdown("Are any of these locking events routinely happening in just a few tables?"))
lock_event_bytable_24_df=lock_event_bytable_24.DataFrame()
lock_event_bytable_24_df.columns=lock_event_bytable_24_df.columns.str.upper()
lock_event_bytable_24_df

---

## Security and Access 

<font color=blue>
The intent is for this section to mature over time and be more of a comprehensive security check. But for now, we are looking at what access PUBLIC has to the database and what ID's may have superuser access.
</font>

In [ ]:
%%sql public_auth <<
SELECT DISTINCT GRANTEE, GRANTEETYPE, 'DATABASE CONNECT AUTHORITY DETECTED' AS ACCESS_LEVEL 
   FROM SYSCAT.DBAUTH 
   WHERE GRANTEE='PUBLIC' AND CONNECTAUTH='Y'
 UNION
SELECT DISTINCT GRANTEE, GRANTEETYPE, 'TABLE LEVEL AUTHORITY DETECTED' AS ACCESS_LEVEL 
   FROM SYSCAT.TABAUTH WHERE GRANTEE='PUBLIC'
   AND TABSCHEMA NOT LIKE 'SYS%'
   AND (CONTROLAUTH='Y' OR ALTERAUTH='Y' OR DELETEAUTH='Y' 
   OR DELETEAUTH='Y' OR INDEXAUTH='Y' OR INSERTAUTH='Y' 
   OR REFAUTH='Y' OR SELECTAUTH='Y' OR UPDATEAUTH='Y')
 UNION
SELECT DISTINCT GRANTEE, GRANTEETYPE, 'PACKAGE LEVEL AUTHORITY DETECTED' AS ACCESS_LEVEL FROM SYSCAT.PACKAGEAUTH WHERE GRANTEE='PUBLIC' AND CONTROLAUTH='Y'
 UNION
SELECT DISTINCT GRANTEE, GRANTEETYPE, 'SCHEMA LEVEL AUTHORITY DETECTED' AS ACCESS_LEVEL FROM SYSCAT.SCHEMAAUTH WHERE GRANTEE='PUBLIC'
   AND SCHEMANAME NOT LIKE 'SYS%'
   AND (ALTERINAUTH='Y' OR CREATEINAUTH='Y' OR DROPINAUTH='Y' 
   OR SELECTINAUTH='Y' OR INSERTINAUTH='Y' OR UPDATEINAUTH='Y' 
   OR DELETEINAUTH='Y' OR EXECUTEINAUTH='Y' OR SCHEMAADMAUTH='Y' 
   OR ACCESSCTRLAUTH='Y' OR DATAACCESSAUTH='Y' OR LOADAUTH='Y')
ORDER BY GRANTEE, GRANTEETYPE, ACCESS_LEVEL
WITH UR

In [ ]:
display(Markdown("####  PUBLIC Access & Authorities"))
display(Markdown("Usually you want to avoid PUBLIC access to anything in the database. Sometimes PUBLIC access is found and is OK (i.e. ability to bind). What we are looking for is the ability for PUBLIC to connect and read or update sensitive information."))
public_auth_df=public_auth.DataFrame()
public_auth_df.columns=public_auth_df.columns.str.upper()
public_auth_df

In [ ]:
%%sql dbadmin_access <<
SELECT GRANTEE AS ID, 'DBADMIN GROUP' AS ACCESS
FROM SYSCAT.DBAUTH
WHERE DBADMAUTH='Y'
UNION ALL
SELECT GRANTEE, 'DATA ACCESS AUTH' AS ACCESS
FROM SYSCAT.DBAUTH
WHERE DATAACCESSAUTH='Y'
ORDER BY ACCESS DESC
WITH UR

In [ ]:
display(Markdown("####  ID's with DBADMIN Access"))
display(Markdown("Are there any ID's that have higher level access than they should? Can they, and should they, have administration or data manipulation privledges."))
dbadmin_access_df=dbadmin_access.DataFrame()
dbadmin_access_df.columns=dbadmin_access_df.columns.str.upper()
dbadmin_access_df

| GROUP OR ACCESS LEVEL | Level of Access |
| :----------- | :----------- |
| DBADMIN Group | Administrative level access with almost unrestricted access to Db2 Commands with the ability to manipulate data. |
| Data Access Auth | Read/Write privledge on all data and tables. |

---

## Error Logs

<font color=blue>
Error logs can tell you so much but are not often spot checked. This section will give you a glimpse of a 3 day period. Db2 can be chatty and put less than critical messages in the error log, but often you can see patterns of behavior or even the unexpected problem if you look closely enough.
</font>

In [ ]:
%%sql error_72hours <<
SELECT 
 TIMESTAMP, 
 DBNAME,
 APPL_ID, 
 LEVEL, 
 IMPACT, 
 SUBSTR(MSG,1, 70) AS MSG 
FROM TABLE (PD_GET_DIAG_HIST( 'MAIN', 'D', '', NULL, NULL) ) AS T 
WHERE LEVEL IN ('C','S','E') 
      AND MSG IS NOT NULL 
      AND TIMESTAMP >= current timestamp - 72 HOURS
ORDER BY TIMESTAMP DESC
WITH UR

In [ ]:
display(Markdown("#### Error Log - Past 72 hours"))
display(Markdown("Data below is not exhaustive. This is showing CRITICAL, SEVERE, or ERROR message in the db2diag.log. "))
error_72hours_df=error_72hours.DataFrame()
error_72hours_df.columns=error_72hours_df.columns.str.upper()
error_72hours_df

---

## Credit & References

- Ragu on Tech: [Db2 Finding Top 10 Most Active Tables](https://www.raghu-on-tech.com/2020/02/29/db2-finding-top-10-most-active-tables/)
- Xtivia: [Index Read Efficiency a KPI for Db2](https://www.virtual-dba.com/blog/index-read-efficiency-db2/)
- Datageek: [Db2 Table Scans](https://datageek.blog/en/2012/11/28/db2-table-scans/)
- DBI Software Blog: [Table Read I/O and Overflows](https://www.dbisoftware.com/blog/db2_performance.php?id=116)
- Xtivia: [Index Read Efficiency - A KPI for Db2](https://www.virtual-dba.com/blog/index-read-efficiency-db2/)
- Datageek: [Boost Your Performance – (IREF) Index Read Efficiency](https://datageek.blog/en/2014/03/17/boost-your-performance-iref-index-read-efficiency/)
- Use the Index Luke: [More Indexes, Slower Insert](https://use-the-index-luke.com/sql/dml/insert)
- Statsology: [How to Create a Pie Chart from Pandas Dataframe](https://www.statology.org/pandas-pie-chart/)
- Mark Gillis, Triton Consulting: IDUG NA D03 Jupyter Notebook Presentation (SQL and Formatting ideas scattered throughout this Notebook.)